# FunSumm

Fine tuning Pegasus with Scisummnet data (NOT what we're planning to do in the project, but trying to run to see what happens)

In [1]:
#Download the dataset
import requests
import io
import zipfile
import pandas as pd
import numpy as np
import os
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
# Importing spacy
import spacy
# Importing json to read input
import json
# Importing rouge for evaluation
from rouge_score import rouge_scorer

import xml.etree.ElementTree as ET
import seaborn as sns
import matplotlib.pyplot as plt
import html
from collections import defaultdict, Counter

sns.set_style("dark")
plot_dims = (16, 16)


# url = "https://cs.stanford.edu/~myasu/projects/scisumm_net/scisummnet_release1.1__20190413.zip"
# response = requests.get(url)
# with zipfile.ZipFile(io.BytesIO(response.content)) as zipObj:
#     # Extract all the contents of zip file in different directory
#     zipObj.extractall("nlp_data")
#     print("File is unzipped in nlp_data folder") 



/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# import pip
# from pip._internal import main as pipmain

# pipmain(['install', 'transformers'])
# pipmain(['install', 'SentencePiece'])

In [3]:
DATA_DIR = "/Users/priscillaburity/Documents/Berkeley/w266/Project/git/mini_10k"

In [4]:
#load our 10k data into a dataframe
papers = []
for root, dirs, files in os.walk(DATA_DIR):
    for f in files:
        fn = root+"/"+f
        with open(fn) as jsonfile:
            d = json.load(jsonfile)
        papers.append(d)

df = pd.DataFrame(papers)
df

,id,guidislink,updated,updated_parsed,published,published_parsed,title,title_detail,summary,summary_detail,...,arxiv_comment,links,arxiv_primary_category,tags,pdf_url,affiliation,arxiv_url,journal_reference,doi,fulltext
0,http://arxiv.org/abs/math/0407145v2,True,2004-07-08T21:21:58Z,"[2004, 7, 8, 21, 21, 58, 3, 190, 0]",2004-07-08T16:10:34Z,"[2004, 7, 8, 16, 10, 34, 3, 190, 0]",Compact packings of the plane with two sizes o...,"{'type': 'text/plain', 'language': None, 'base...",We consider packings of the plane using discs ...,"{'type': 'text/plain', 'language': None, 'base...",...,"Latex, 20 pages, 15 postscript figures Only ch...",[{'href': 'http://arxiv.org/abs/math/0407145v2...,"{'term': 'math.MG', 'scheme': 'http://arxiv.or...","[{'term': 'math.MG', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/math/0407145v2,None,http://arxiv.org/abs/math/0407145v2,"Discrete and Computational Geometry 35 (2006),...",None,arXiv:math/0407145v2 [math.MG] 8 Jul 2004\n\nC...
1,http://arxiv.org/abs/1007.0906v1,True,2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",Matrix Algebras and Semidefinite Programming T...,"{'type': 'text/plain', 'language': None, 'base...",This PhD thesis is concerned with SDP bounds f...,"{'type': 'text/plain', 'language': None, 'base...",...,"PhD Thesis (2005), about 100 pages","[{'href': 'http://arxiv.org/abs/1007.0906v1', ...","{'term': 'math.CO', 'scheme': 'http://arxiv.or...","[{'term': 'math.CO', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1007.0906v1,None,http://arxiv.org/abs/1007.0906v1,None,None,arXiv:1007.0906v1 [math.CO] 6 Jul 2010\n\nMatr...
2,http://arxiv.org/abs/physics/0208084v1,True,2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",Resonant Absorption between Moving Atoms due t...,"{'type': 'text/plain', 'language': None, 'base...",By taking both the Doppler frequency shift for...,"{'type': 'text/plain', 'language': None, 'base...",...,None,[{'href': 'http://arxiv.org/abs/physics/020808...,"{'term': 'physics.gen-ph', 'scheme': 'http://a...","[{'term': 'physics.gen-ph', 'scheme': 'http://...",http://arxiv.org/pdf/physics/0208084v1,None,http://arxiv.org/abs/physics/0208084v1,None,None,arXiv:physics/0208084v1 [physics.gen-ph] 23 Au...
3,http://arxiv.org/abs/1112.2113v1,True,2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",Incremental Slow Feature Analysis: Adaptive an...,"{'type': 'text/plain', 'language': None, 'base...",Slow Feature Analysis (SFA) extracts features ...,"{'type': 'text/plain', 'language': None, 'base...",...,None,"[{'href': 'http://arxiv.org/abs/1112.2113v1', ...","{'term': 'cs.AI', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",http://arxiv.org/pdf/1112.2113v1,None,http://arxiv.org/abs/1112.2113v1,"Neural Computation, 2012, Vol. 24, No. 11, Pag...",None,Incremental Slow Feature Analysis: Adaptive an...
4,http://arxiv.org/abs/1106.0641v1,True,2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",Interval Semirings,"{'type': 'text/plain', 'language': None, 'base...",This book has seven chapters. In chapter one w...,"{'type': 'text/plain', 'language': None, 'base...",...,155 pages; Published by Kappa & Omega in 2011,"[{'href': 'http://arxiv.org/abs/1106.0641v1', ...","{'term': 'math.GM', 'scheme': 'http://arxiv.or...","[{'term': 'math.GM', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1106.0641v1,None,http://arxiv.org/abs/1106.0641v1,None,None,INTERVAL SEMIRINGS\n\nW. B. Vasantha Kandasamy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://arxiv.org/abs/1010.3242v1,True,2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",5-qubit quantum error correct

In [5]:
# #get all raw text, break all papers into two parts -- Abstract and rest of document
# #first get all filepaths
# abstracts = []
# body = []
# citations = []
# for subdir, dirs, files in os.walk(DATA_DIR):
#     for filename in files:
#         filepath = subdir + os.sep + filename
#         if filepath.startswith("body_"):
#             body.append(filepath)
#         if filepath.startswith("abstracts"):
#             abstracts.append(filepath)
#         if filepath.endswith(".CS"):
#             citations.append(filepath)

In [6]:
#next parse all XML documents

def parse_xml_abstract(fp):
    """ parse an XML journal article into an abstract and the rest of the text
    """
    try:
        tree = ET.parse(fp)
    except Exception as e:
        return "","",str(e)
    root = tree.getroot()
    
    ab = []
    bod = []
    
    for child in root:
        if child.tag == "ABSTRACT":
            for block in child:
                ab.append(block.text)
        else:
            for block in child:
                bod.append(block.text)
                
    #convert from list --> string
    abstract = "\n".join(ab)
    body = "\n".join(bod)
    
    #decode html entities
    abstract = html.unescape(abstract)
    body = html.unescape(body)
    
    return abstract,body,""




In [7]:
# raw_cols = []
# for fpn in range(len(xmlfiles)):
#     ab,bod,err = parse_xml_abstract(xmlfiles[fpn])
#     if err:
#         #print(fp, err)
#         continue
#     f = open(citations[fpn]) 

#     # returns JSON object as  
#     # a dictionary 
#     data = json.load(f) 
#     only_text = []
#     for entry in data:
#         only_text.append(entry['clean_text'])
# #     print(only_text)
        
#     raw_cols.append([ab,bod,only_text,xmlfiles[fpn]])

# df = pd.DataFrame(raw_cols, columns=["abstract","body","citations", "filepath"])
# df

In [8]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-arxiv')
# The PEGASUS Model with a language modeling head. Can be used for summarization. 
# This model inherits from PreTrainedModel. 

tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-arxiv')

# ARTICLE_TO_SUMMARIZE = (
# "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
# "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
# "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
# )


In [9]:
## Sample with no fine-tunning 
sample_id = 100
sample_input = df.fulltext[sample_id]
inputs = tokenizer([sample_input], max_length=1024, return_tensors='pt')
sample_input = tokenizer.decode(inputs['input_ids'][0]).replace('⁇', '')
inputs = tokenizer([sample_input], max_length=1024, return_tensors='pt')

#'max_length': Pad to a maximum length specified with the argument max_length 
# or to the maximum acceptable input length for the model if that argument is not provided.

# Generate Summary
summary_ids = model.generate(inputs['input_ids'])
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

sample_output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
print(sample_output)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['in this paper , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 ) . in this paper <n> , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 ) . in this paper <n> , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 ) . in this paper <n> , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 ) . in this paper <n> , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 ) . in this paper <n> , we introduce and study the concept of an induced extension of a ring . <n> an induced extension of a ring was first introduced by ariv ( 2006 )']


In [10]:
sample_reference = df.summary[sample_id ]
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(str(sample_input),
                      str(sample_output))
print(scores)

{'rouge1': Score(precision=0.6089385474860335, recall=0.1607669616519174, fmeasure=0.25437572928821467), 'rougeL': Score(precision=0.4022346368715084, recall=0.10619469026548672, fmeasure=0.1680280046674446)}


## Fine-tune with custom data

In [11]:
import pandas as pd
# in_df = pd.read_csv('/content/drive/My Drive/summaries_sample.csv')

df.rename(columns={"fulltext": "allTextReprocess"}, inplace = True)
df.rename(columns={"summary": "summaries"}, inplace = True)

in_df = df
in_df

,id,guidislink,updated,updated_parsed,published,published_parsed,title,title_detail,summaries,summary_detail,...,arxiv_comment,links,arxiv_primary_category,tags,pdf_url,affiliation,arxiv_url,journal_reference,doi,allTextReprocess
0,http://arxiv.org/abs/math/0407145v2,True,2004-07-08T21:21:58Z,"[2004, 7, 8, 21, 21, 58, 3, 190, 0]",2004-07-08T16:10:34Z,"[2004, 7, 8, 16, 10, 34, 3, 190, 0]",Compact packings of the plane with two sizes o...,"{'type': 'text/plain', 'language': None, 'base...",We consider packings of the plane using discs ...,"{'type': 'text/plain', 'language': None, 'base...",...,"Latex, 20 pages, 15 postscript figures Only ch...",[{'href': 'http://arxiv.org/abs/math/0407145v2...,"{'term': 'math.MG', 'scheme': 'http://arxiv.or...","[{'term': 'math.MG', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/math/0407145v2,None,http://arxiv.org/abs/math/0407145v2,"Discrete and Computational Geometry 35 (2006),...",None,arXiv:math/0407145v2 [math.MG] 8 Jul 2004\n\nC...
1,http://arxiv.org/abs/1007.0906v1,True,2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",Matrix Algebras and Semidefinite Programming T...,"{'type': 'text/plain', 'language': None, 'base...",This PhD thesis is concerned with SDP bounds f...,"{'type': 'text/plain', 'language': None, 'base...",...,"PhD Thesis (2005), about 100 pages","[{'href': 'http://arxiv.org/abs/1007.0906v1', ...","{'term': 'math.CO', 'scheme': 'http://arxiv.or...","[{'term': 'math.CO', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1007.0906v1,None,http://arxiv.org/abs/1007.0906v1,None,None,arXiv:1007.0906v1 [math.CO] 6 Jul 2010\n\nMatr...
2,http://arxiv.org/abs/physics/0208084v1,True,2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",Resonant Absorption between Moving Atoms due t...,"{'type': 'text/plain', 'language': None, 'base...",By taking both the Doppler frequency shift for...,"{'type': 'text/plain', 'language': None, 'base...",...,None,[{'href': 'http://arxiv.org/abs/physics/020808...,"{'term': 'physics.gen-ph', 'scheme': 'http://a...","[{'term': 'physics.gen-ph', 'scheme': 'http://...",http://arxiv.org/pdf/physics/0208084v1,None,http://arxiv.org/abs/physics/0208084v1,None,None,arXiv:physics/0208084v1 [physics.gen-ph] 23 Au...
3,http://arxiv.org/abs/1112.2113v1,True,2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",Incremental Slow Feature Analysis: Adaptive an...,"{'type': 'text/plain', 'language': None, 'base...",Slow Feature Analysis (SFA) extracts features ...,"{'type': 'text/plain', 'language': None, 'base...",...,None,"[{'href': 'http://arxiv.org/abs/1112.2113v1', ...","{'term': 'cs.AI', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",http://arxiv.org/pdf/1112.2113v1,None,http://arxiv.org/abs/1112.2113v1,"Neural Computation, 2012, Vol. 24, No. 11, Pag...",None,Incremental Slow Feature Analysis: Adaptive an...
4,http://arxiv.org/abs/1106.0641v1,True,2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",Interval Semirings,"{'type': 'text/plain', 'language': None, 'base...",This book has seven chapters. In chapter one w...,"{'type': 'text/plain', 'language': None, 'base...",...,155 pages; Published by Kappa & Omega in 2011,"[{'href': 'http://arxiv.org/abs/1106.0641v1', ...","{'term': 'math.GM', 'scheme': 'http://arxiv.or...","[{'term': 'math.GM', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1106.0641v1,None,http://arxiv.org/abs/1106.0641v1,None,None,INTERVAL SEMIRINGS\n\nW. B. Vasantha Kandasamy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://arxiv.org/abs/1010.3242v1,True,2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",5-qubit quantum err

In [14]:
# clean for math symbols

for index, row in df.iterrows():
    sample_input = row['allTextReprocess']
    inputs = tokenizer([sample_input], max_length=1024, return_tensors='pt')
    sample_input = tokenizer.decode(inputs['input_ids'][0]).replace('⁇', '')
    row['allTextReprocess'] = sample_input
    
    
    sample_input = row['summaries']
    inputs = tokenizer([sample_input], max_length=1024, return_tensors='pt')
    sample_input = tokenizer.decode(inputs['input_ids'][0]).replace('⁇', '')
    row['summaries'] = sample_input

    if index % 500 == 0:
        print(index)
        




0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [ ]:
print(index)
df.shape[0]

In [15]:
# Train Test Split
train_pct = 0.6
test_pct = 0.2

in_df = in_df.sample(len(in_df), random_state=20)
train_sub = int(len(in_df) * train_pct)
test_sub = int(len(in_df) * test_pct) + train_sub

train_df = in_df[0:train_sub]
test_df = in_df[train_sub:test_sub]
val_df = in_df[test_sub:]

train_texts = list(train_df['allTextReprocess'])
test_texts = list(test_df['allTextReprocess'])
val_texts = list(val_df['allTextReprocess'])

train_decode = list(train_df['summaries'])
test_decode = list(test_df['summaries'])
val_decode = list(val_df['summaries'])

In [16]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [17]:
train_decodings = tokenizer(train_decode, truncation=True, padding=True)
val_decodings = tokenizer(val_decode, truncation=True, padding=True)
test_decodings = tokenizer(test_decode, truncation=True, padding=True)

In [20]:
# for key, val in test_decodings.items():
#     print(key)
len(train_decodings)    

2

In [18]:
import torch

class ourDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, decodings):
        self.encodings = encodings
        self.decodings = decodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.decodings['input_ids'][idx])  # torch.tensor(self.labels[idx])
#         print(item)
        return item

    def __len__(self):
        return len(self.encodings)

train_dataset = ourDataset(train_encodings, train_decodings)
val_dataset = ourDataset(val_encodings, val_decodings)
test_dataset = ourDataset(test_encodings, test_decodings)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset
    eval_dataset=val_dataset         # evaluation dataset
)

trainer.train()

In [ ]:
#trainer.evaluate()

In [ ]:
trainer.save_model('fine-tuned')

In [ ]:
model_fine_tuned = PegasusForConditionalGeneration.from_pretrained('fine-tuned')

In [ ]:
## Sample with fine-tunning 

sample_input = df.allTextReprocess[0]
inputs = tokenizer([sample_input], max_length=1024, return_tensors='pt')

#'max_length': Pad to a maximum length specified with the argument max_length 
# or to the maximum acceptable input length for the model if that argument is not provided.

# Generate Summary
summary_ids = model_fine_tuned.generate(inputs['input_ids'])
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

sample_output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
print(sample_output)

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores_fine_tuned = scorer.score(str(sample_input),
                      str(sample_output))
print(scores_fine_tuned)
print(scores)

In [ ]:
# !python3 -c 'import tensorflow as tf; print(tf.__version__)' 